In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from fontTools.ttLib import TTFont
from PIL import Image, ImageDraw, ImageFont
from tensorflow.python.lib.io.file_io import FileIO

2024-02-06 12:23:21.213600: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 12:23:21.213695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 12:23:21.363571: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 12:23:21.663449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def get_existing_chars(filename):
    ttfont = TTFont(filename)
    return list({chr(key) for table in ttfont['cmap'].tables for key in table.cmap.keys()})

In [5]:
def get_image(font, char):
    img = Image.new('L', (500,500), 255)
    
    draw = ImageDraw.Draw(img)
    draw.text((0,0), char, font=font)
    
    npimg = 255 - np.array(img)
    wmin = npimg.sum(0).nonzero()[0].min()
    wmax = npimg.sum(0).nonzero()[0].max()
    hmin = npimg.sum(1).nonzero()[0].min()
    hmax = npimg.sum(1).nonzero()[0].max()
    npimg = 255 - npimg[hmin:hmax+1,wmin:wmax+1]
    return Image.fromarray(npimg)
    # return npimg

In [29]:
# utf-8
def get_all_korean():
    
    def nextKorLetterFrom(letter):
        lastLetterInt = 15572643
        if not letter:
            return '가'
        a = letter
        b = a.encode('utf8')
        c = int(b.hex(), 16)

        if c == lastLetterInt:
            return False

        d = hex(c + 1)
        e = bytearray.fromhex(d[2:])

        flag = True
        while flag:
            try:
                r = e.decode('utf-8')
                flag = False
            except UnicodeDecodeError:
                c = c+1
                d = hex(c)
                e = bytearray.fromhex(d[2:])
        return e.decode()
    
    returns = []
    flag = True
    k = ''
    while flag:
        k = nextKorLetterFrom(k)
        if k is False:
            flag = False
        else:
            returns.append(k)
    return returns
    

In [30]:
ak = get_all_korean()

In [31]:
len(ak)

11172

In [32]:
fd = "data/r40202/ttfs/"
ttfs = [(fd+"apple/"+f,"apple", f.split(".ttf")[0]) for f in os.listdir(fd+"apple")]

In [33]:
ttfs

[('data/r40202/ttfs/apple/AppleMyungjo.ttf', 'apple', 'AppleMyungjo')]

In [34]:
png_fd = "data/r40202/pngs/"

In [35]:
ttf

('data/r40202/ttfs/apple/AppleMyungjo.ttf', 'apple', 'AppleMyungjo')

In [37]:
ttf = ttfs[0]
filename = ttf[0]
font = ImageFont.truetype(filename,255)
chardf = pd.DataFrame(ak)
chardf.to_parquet(png_fd+"%s__%s.parquet"%(ttf[1],ttf[2]))
for i in tqdm(range(len(chardf))):
    char = chardf.iloc[i].values[0]
    try:
        img = get_image(font, char)
        with open(png_fd+'%s__%s__%s.png'%(ttf[1],ttf[2],str(i)), "wb") as f:
            img.save(f, "PNG")
    except:
        # print(ttf[1], ttf[2], char)
        pass


100%|██████████| 11172/11172 [00:38<00:00, 286.59it/s]


In [61]:
applemj = [f for f in os.listdir(png_fd) if "apple__AppleMyungjo__" in f]

In [62]:
len(applemj)

11172

In [51]:
len(chardf)

11172